In [ ]:
# Copyright 2018 Analytics Zoo Authors.
# http://www.apache.org/licenses/LICENSE-2.0

import warnings
warnings.filterwarnings("ignore", message = "numpy.dtype size changed")

import tensorflow as tf
from zoo import init_nncontext
from zoo.pipeline.api.net import TFOptimizer, TFDataset
from bigdl.optim.optimizer import *
import sys
# from tensorflow.keras.models import Model, Sequential
# from tensorflow.keras.layers import *
from zoo.pipeline.api.keras.models import Sequential
from zoo.pipeline.api.keras.layers import Dense, Activation, Flatten, Reshape, Convolution2D, MaxPooling2D

from bigdl.dataset import mnist
from bigdl.dataset.transformer import *

In [ ]:
sc = init_nncontext()

In [ ]:
# train_rdd = sc.parallelize(mnist.read_data_sets("/tmp/mnist", "train")) \
#             .map(lambda x: read_image_and_label(x)) \
#             .map(lambda image_label: decode_to_ndarrays(image_label))

In [ ]:
# model = Sequential()
# model.add(Reshape((1, 28, 28), input_shape=(28, 28, 1)))
# model.add(Convolution2D(6, 5, 5, activation="tanh", name="conv1_5x5"))
# model.add(MaxPooling2D())
# model.add(Convolution2D(12, 5, 5, activation="tanh", name="conv2_5x5"))
# model.add(MaxPooling2D())
# model.add(Flatten())
# model.add(Dense(100, activation="tanh", name="fc1"))
# model.add(Dense(10, activation="softmax", name="fc2"))

In [ ]:
# model.compile(loss = 'mean_squared_error', optimizer = 'sgd', metrics = ['mae'])

In [ ]:
# model.get_input_shape()

In [ ]:
# model.get_output_shape()

In [ ]:
# model.fit(training_rdd, batch_size = 128, nb_epoch = 5)

In [ ]:
# get data, pre-process and create TFDataset
def get_data_rdd(dataset):
    (images_data, labels_data) = mnist.read_data_sets("/tmp/mnist", dataset)
    # image_rdd = sc.parallelize(images_data[:data_num])
    image_rdd = sc.parallelize(images_data)
    # labels_rdd = sc.parallelize(labels_data[:data_num])
    labels_rdd = sc.parallelize(labels_data)
    rdd = image_rdd.zip(labels_rdd) \
        .map(lambda rec_tuple: [normalizer(rec_tuple[0], mnist.TRAIN_MEAN, mnist.TRAIN_STD),
                                np.array(rec_tuple[1])])
    return rdd

In [ ]:
# data_num = 60000
training_rdd = get_data_rdd("train")
testing_rdd = get_data_rdd("test")

batch_size = 512

# Images are 28 by 28 pixels with one color channel for black and white; labels have no dimmensions because they are integers
dataset = TFDataset.from_rdd(training_rdd,
                             names=["features", "labels"],
                             shapes=[[28, 28, 1], []], 
                             types=[tf.float32, tf.int32],
                             batch_size=batch_size,
                             val_rdd=testing_rdd
                             )

In [ ]:
training_rdd.take(3)

In [ ]:
data = Input(shape=[28, 28, 1]) # Must match the dimmensions of the images
x = Flatten()(data)
x = Dense(64, activation='relu')(x)
x = Dense(64, activation='relu')(x)
predictions = Dense(10, activation='softmax')(x)

model = Model(inputs=data, outputs=predictions)

model.compile(optimizer='rmsprop',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

optimizer = TFOptimizer.from_keras(keras_model=model, dataset=dataset)

In [ ]:
optimizer.set_train_summary(TrainSummary("/tmp/mnist_log", "mnist"))
optimizer.set_val_summary(ValidationSummary("/tmp/mnist_log", "mnist"))
# kick off training
max_epoch = 5
optimizer.optimize(end_trigger=MaxEpoch(max_epoch))

# model.save_weights("/tmp/mnist_keras.h5")